In [ ]:
import numpy as np
from scipy import signal
import control as ct
import matplotlib.pyplot as plt

from spectools import lpsd, dsp
lpsd.use_c_core = False

import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

In [ ]:
# Step 1: Define the system transfer function
tau = 100  # Time constant
fs = 2  # Sampling frequency (Hz)
dt = 1 / fs  # Time step
n = int(2e6)  # Number of samples

# Transfer function H(s) = 1 / (tau*s + 1)
num = [1]
den = [tau, 1]
system = signal.TransferFunction(num, den)

# Step 2: Generate the input signal (white noise)
np.random.seed(42)  # For reproducibility
input_signal = np.random.normal(0, 1, n)

# Step 3: Generate the output signal by filtering the input through the system
t = np.linspace(0, n*dt, n)
_, output_signal, _ = signal.lsim(system, input_signal, t)

# Introduce frequency-dependent noise to the output
freq = np.fft.fftfreq(n, d=dt)
noise = np.fft.ifft(np.fft.fft(np.random.normal(0, 1, n)) * (np.abs(freq)**0.5)).real
output_signal_noisy = output_signal + 0.5 * noise  # Scale noise appropriately

In [ ]:
fig, ax = plt.subplots()
ax.plot(input_signal, label='input')
ax.plot(output_signal_noisy, label='output')
ax.set_xlabel("Time (s)")
ax.set_ylabel("Signals")
ax.legend()
ax.set_xlim(0,1000)
plt.show()

In [ ]:
# Step 4: Compute the cross-spectral density (Welch)
f, Pxy = signal.csd(input_signal, output_signal_noisy, fs=fs, nperseg=10000)

In [ ]:
# Step 5: Compute the cross-spectral density (spectools)
ltf_obj = lpsd.ltf([input_signal, output_signal_noisy], fs=fs, pool=pool)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.semilogx(f, ct.mag2db(np.abs(Pxy)), ls='--', label='Welch')
ax2.semilogx(f, -np.angle(Pxy, deg=True),  ls='--')
ax2.semilogx(ltf_obj.f, np.angle(ltf_obj.Gxy, deg=True))
ax1.semilogx(ltf_obj.f, ct.mag2db(np.abs(ltf_obj.Gxy)), label='spectools')
ax2.set_xlabel("Frequency (Hz)")
ax1.set_ylabel("Magnitude (dB)")
ax2.set_ylabel("Phase (deg)")
ax1.set_title("Cross spectral density estimate")
ax1.legend(framealpha=1, edgecolor='k')
ax1.grid(ls='--')
ax2.grid(ls='--')
ax1.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax2.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
plt.show()

In [ ]:
fig, (ax2, ax1) = ltf_obj.plot('bode', errors=True, sigma=3)
ax1.set_ylim(-120, 60)

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(ltf_obj.f, ltf_obj.coh, label='Coherence')
ax.semilogx(ltf_obj.f, ltf_obj.coh_error, label='Normalized random error')
ax.semilogx(ltf_obj.f, ltf_obj.coh_dev, label='Standard deviation')
ax.semilogx(ltf_obj.f, ltf_obj.coh_error*ltf_obj.coh, ls=':')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel("Coherence")
ax.grid(ls='--')
ax.legend(framealpha=1, edgecolor='k')
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax.set_ylim(0,1.2)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(ltf_obj.f, ltf_obj.coh_dev, lw=6, c='gray', label=r'$\sigma[\gamma_{xy}^2]$')
ax.loglog(ltf_obj.f, ltf_obj.coh_error*ltf_obj.coh, c='k', label=r'$ \gamma_{xy}^2 \cdot \epsilon_r [\gamma_{xy}^2]$')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax.grid(ls='--')
ax.legend(edgecolor='k', framealpha=1)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(ltf_obj.f, ltf_obj.Gxy_dev, lw=6, c='gray', label=r'$\sigma[ |G_{xy}| ]$')
ax.loglog(ltf_obj.f, ltf_obj.Gxy_error*np.abs(ltf_obj.Gxy), c='k', label=r'$|G_{xy}| \cdot \epsilon_r[|G_{xy}|]$')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax.legend(edgecolor='k', framealpha=1)
ax.grid(ls='--')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(ltf_obj.f, ltf_obj.Hxy_dev, lw=6, c='gray', label=r'$\sigma[ |H_{xy}| ]$')
ax.loglog(ltf_obj.f, ltf_obj.cf_mag_error*ltf_obj.cf, c='k', label=r'$|H_{xy}| \cdot \epsilon_r[|H_{xy}|]$')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax.legend(edgecolor='k', framealpha=1)
ax.grid(ls='--')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(ltf_obj.f, ltf_obj.cf_rad_error, lw=6, c='gray', label=r'$\epsilon_r[ arg[ H_{xy}] ]$')
ax.loglog(ltf_obj.f, ltf_obj.Hxy_dev/ltf_obj.cf, c='k', label=r'$\sigma[|H_{xy}|] / |H_{xy}|$')
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel(r"$\sigma(\gamma_{xy}^2)$")
ax.set_xlim(ltf_obj.f[0],ltf_obj.f[-1])
ax.legend(edgecolor='k', framealpha=1)
ax.grid(ls='--')
plt.show()